In [1]:
from bpm_detection import get_bpm
from pyrubberband.pyrb import time_stretch
from scipy.io.wavfile import read, write
from scipy.signal import resample
import numpy as np

In [2]:
def normalise_float32(x):
    x = np.float32(x)
    x = x - np.min(x)
    m = np.max(x)
    if m > 0:
        x = x / m
    x = x * 2 - 1
    return x

'''
Converts npy-array of range [-1.0, +1.0] back to int16 (-32768 bis 32767)
'''
def to_int16(x):
    return np.int16(x * 32767)

def norm_audio(x):
    return to_int16(normalise_float32(x))

In [3]:
target_sr = 44100
target_bpm = 140
filename = "/PATH/TO/MY/WAVFILE.wav"

In [5]:
# Assure sample rate
sr, audio = read(filename)
audio = norm_audio(audio)
if sr != target_sr:
    print('resampling from ', sr, ' --> ', target_sr)
    target_num_samples = int(round(len(audio) / sr * target_sr))
    audio = norm_audio(resample(audio, target_num_samples))

# Write audio as tmp.wav (take care, because of this action the script can't be parallelised without modification)
write("tmp.wav", target_sr, audio)

# get bpm
bpm = get_bpm("tmp.wav", window=30)
print(bpm)

resampling from  48000  -->  44100
Completed!  Estimated Beats Per Minute: 127.01612903225806
127.01612903225806


In [6]:
# do timestretch
sr, audio = read("tmp.wav")
dur_track = len(audio) / sr / 60 # in minutes
num_beats = dur_track * bpm
target_dur = num_beats / target_bpm
rate = dur_track / target_dur
print(dur_track, ' --> ', target_dur, ' rate: ', rate)
audio_timestretched = time_stretch(audio, sr, rate, rbargs=None)

# write stretched file
write("stretched.wav", sr, audio_timestretched)

4.0  -->  3.629032258064516  rate:  1.1022222222222222


In [7]:
# check back bpm
bpm = get_bpm("stretched.wav", window=30)
print(bpm)

Completed!  Estimated Beats Per Minute: 139.97037663986458
139.97037663986458
